In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Lecture 9 Exercise 1

What is the total frequency search range for L2 and L5 carrier, if the TCXO has the following specifications:

TCXO offset within +-3ppm

In [ ]:
tcxo_offset = 3e-6
f_L1 = 1575.42e6
f_L2 = 1227.60e6
f_L5 = 1176.45e6
f_L1_offset = f_L1 * tcxo_offset
f_L2_offset = f_L2 * tcxo_offset
f_L5_offset = f_L5 * tcxo_offset
print(f"L5 Doppler search range: {-f_L5_offset} Hz to {f_L5_offset} Hz")
print(f"L2 Doppler search range: {-f_L2_offset} Hz to {f_L2_offset} Hz")
print(f"L1 Doppler search range: {-f_L1_offset} Hz to {f_L1_offset} Hz")

# Lecture 9 Exercise 2

How long does it take for an L2C code and an L5 code to mis-align by ½ chip due to Doppler frequency?

In [ ]:
code_rate_L2C = 1.023e6  # chips per second
code_rate_L5 = 10.23e6  # chips per second

f_L1 = 1575.42e6
f_L2 = 1227.60e6
f_L5 = 1176.45e6
speed_of_light = 299792458  # meters per second
v_LOS_eff = speed_of_light * tcxo_offset  # effective TCXO rate in m/s

num_steps = 1000
time_rate_offset = np.linspace(-tcxo_offset, tcxo_offset, num_steps)  # Hz
doppler_arr_L2C = f_L2 * time_rate_offset
doppler_arr_L5 = f_L5 * time_rate_offset

misalignment_amount = 0.5
time_to_half_chip_L2C = misalignment_amount / code_rate_L2C / time_rate_offset
time_to_half_chip_L5 = misalignment_amount / code_rate_L5 / time_rate_offset
time_to_half_chip_L2C[num_steps // 2 - 1: num_steps // 2 + 1] = np.nan  # avoid division by zero
time_to_half_chip_L5[num_steps // 2 - 1: num_steps // 2 + 1] = np.nan  # avoid division by zero

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(doppler_arr_L2C, time_to_half_chip_L2C * 1e3, label="L2C Code", color="blue")
ax.plot(doppler_arr_L5, time_to_half_chip_L5 * 1e3, label="L5 Code", color="orange")
ax.set_xlabel("Doppler Frequency [Hz]")
ax.set_ylabel("Time to ½ Chip Misalignment [ms]")
ax.set_ylim(-2e3, 2e3)
ax.set_title("Time to ½ Chip Misalignment vs Doppler Frequency")
ax.legend()
ax.grid()
plt.show()

# Lecture 9 Exercise 3

What is the computation cost (in terms of number of multiplications and additions) of time domain correlation of 1 ms of GPS L1 CA signal, assuming that the signal is sampled at 5 MHz.

(TODO: This exercise needs to be re-written to specify the number of code phase and Doppler bins searched.)

Assume that our received signal samples and replica signal are stored in memory.  We ignore memory access time and focus only on the number of arithmetic operations.

For $N$ samples, we need $N$ multiplications and $N-1$ additions to compute the correlation value for one code phase.  This result is $\mathcal{O}(N)$.

If we want to compute the correlation for all possible code phases, we need to repeat this for each code phase of interest.  Since 1 ms is equal to one CA code period, and the sampling frequency is 5 MHz, we have $N = 5000$ samples in 1 ms.  If we check each possible code phase at the resolution corresponding to this sampling rate ($\Delta\eta = 1.023\cdot 10^6 / f_\text{samp} \approx 0.2$ chips), we have $N$ code phases to check.

In [ ]:
samp_rate = 5e6
chip_rate = 1.023e6
code_phase_resolution = chip_rate / samp_rate
print(f"Code phase resolution: {code_phase_resolution} chips")

# Lecture 9 Exercise 4 and Lecture 11 Exercise 1
Note: Lecture 11 used to be numbered as Lecture 10

Implement a function that performs acquisition using the time domain correlation method.  You need to demonstrate that your program works with at least 3 sets of data. For all 3 sets of data, f0 = 1.25e6, and sampling frequency is 5MHZ, bit resolution is 4-bit. Try using the following parameters in your simulated signal: PRN =[4 7 10 15];   CN0 =[45 47 49 45]; n0 = [100 1225 2500 4999];     Initial phase phi = [0 0.5 1.2 1.34] radians;   fd = [1000 1000 2200 3000] Hz;​

See `lecture-8/simulate-gps-signal.ipynb` for reference on how to simulate the signals.

Here we use a simpler linear signal model to simulate the received signal.

The original exercise wording uses code phase in units of samples, denoted as `n0`.  In this notebook, we use code phase in units of seconds or chips, denoted as `eta0`.  Conversion between the two more generally depends on whether Doppler expansion/compression of the code is taken into account.  Here, since we are considering L1 signals with < 5 kHz Doppler, we can ignore this effect for simplicity.

```python
fs = 5e6  # samples per second
chip_rate = 1.023e6  # chips per second
n0 = 2500  # code phase in samples
eta0 = n0 / fs  # converts code phase from samples to seconds
eta0_chips = eta0 * chip_rate  # converts code phase from seconds to chips
```

We implement both the time-domain (Lecture 9 Exercise 4) and frequency-domain (Lecture 10 Exercise 1) correlation methods below.  You can uncomment the frequency-domain correlation code to test it out.  By default, we assume we perform a single 1 ms coherent correlation (M=1).

In [ ]:
import numpy as np
from dataclasses import dataclass
import utils.signals.gps_l1ca as gps_l1ca_utils

@dataclass
class SignalParams:
    prn: int
    cn0: float  # in dB-Hz
    n0: int     # initial sample code phase
    initial_phase: float  # in radians
    fd: float   # Doppler frequency in Hz

signal_params = {
    4: SignalParams(prn=4, cn0=45, n0=100, initial_phase=0.0, fd=1000),
    7: SignalParams(prn=7, cn0=47, n0=1225, initial_phase=0.5, fd=1000),
    10: SignalParams(prn=10, cn0=49, n0=2500, initial_phase=1.2, fd=2200),
    15: SignalParams(prn=15, cn0=45, n0=4999, initial_phase=1.34, fd=3000),
}

samp_rate = 5e6  # 5 MHz
sim_duration = 0.1  # seconds
sim_time_arr = np.arange(0, sim_duration, 1.0 / samp_rate)
sim_num_samples = len(sim_time_arr)

# First, compute the noise power based on receiver equivalent noise temperature and front-end bandwidth
k_boltzmann = 1.380649e-23  # J/K
rx_sys_temp_k = 535.0  # Kelvin
front_end_bw_hz = samp_rate  # Hz
noise_power_watts = k_boltzmann * rx_sys_temp_k * front_end_bw_hz

noise_samples = np.sqrt(noise_power_watts) * (
    np.random.randn(sim_num_samples) + 1j * np.random.randn(sim_num_samples)
) / np.sqrt(2)

noise_free_signal_samples = np.zeros(sim_num_samples, dtype=complex)

for prn, params in signal_params.items():
    sat_cn0_dBHz = params.cn0
    signal_power = noise_power_watts / front_end_bw_hz * 10**(sat_cn0_dBHz / 10)
    signal_amp = np.sqrt(signal_power)

    code_seq = 1 - 2 * gps_l1ca_utils.generate_code_sequence_L1CA(prn).astype(np.int8)
    eta0 = params.n0 / samp_rate  # initial code phase in seconds
    code_phase_chips = (eta0 + sim_time_arr) * gps_l1ca_utils.CODE_RATE
    code_chip_indices = (code_phase_chips).astype(int) % gps_l1ca_utils.CODE_LENGTH
    code_samples = code_seq[code_chip_indices]

    num_data_bits = int(np.ceil(sim_duration * gps_l1ca_utils.DATA_SYMBOL_RATE))
    data_bits = np.random.choice([-1, 1], size=num_data_bits)
    data_bit_indices = ((eta0 + sim_time_arr) * gps_l1ca_utils.DATA_SYMBOL_RATE).astype(int) % num_data_bits
    data_samples = data_bits[data_bit_indices]

    carrier_phase = 2 * np.pi * (params.fd * sim_time_arr + params.initial_phase / (2 * np.pi))
    carrier_samples = np.exp(1j * carrier_phase)

    signal_samples = signal_amp * code_samples * data_samples * carrier_samples
    noise_free_signal_samples += signal_samples

baseband_samples = noise_free_signal_samples + noise_samples

num_quant_bits = 4
num_quant_levels = 2**num_quant_bits
# assume even quantization levels between +/- max_signal_amp
#  (sometimes this quantization doesn"t quite work as expected; it"s not that important though -- it should be close enough)
max_signal_amp = np.max(np.abs(baseband_samples))
quantized_samples_real = np.floor(np.real(baseband_samples) / max_signal_amp * (num_quant_levels / 2))
quantized_samples_imag = np.floor(np.imag(baseband_samples) / max_signal_amp * (num_quant_levels / 2))
quantized_samples = quantized_samples_real + 1j * quantized_samples_imag

In [ ]:
def delay_doppler_correlate_gps_l1ca_signal(
        received_samples: np.ndarray,
        prn: int,
        min_doppler_hz: float,
        max_doppler_hz: float,
        samp_rate: float,
        doppler_step_hz: float | None = None,
        print_progress: bool = True,
) -> None:
    code_seq = 1 - 2 * gps_l1ca_utils.generate_code_sequence_L1CA(prn).astype(np.int8)
    num_samples = len(received_samples)
    code_length = gps_l1ca_utils.CODE_LENGTH
    code_rate = gps_l1ca_utils.CODE_RATE

    if doppler_step_hz is None:
        doppler_step_hz = samp_rate / num_samples  # 1 / T resolution
    doppler_arr_hz = np.arange(min_doppler_hz, max_doppler_hz + doppler_step_hz, doppler_step_hz)
    num_doppler_bins = len(doppler_arr_hz)

    time_arr = np.arange(num_samples) / samp_rate
    code_seq = 1 - 2 * gps_l1ca_utils.get_GPS_L1CA_code_sequence(prn).astype(np.int8)
    code_phase_chips = time_arr * code_rate  # Assume no Doppler expansion/compression
    code_chip_indices = (code_phase_chips).astype(int) % code_length
    code_samples = code_seq[code_chip_indices]
    num_code_phases = len(code_samples)

    # Compute FFT of code samples for frequency-domain correlation
    # code_samples_fft = np.fft.fft(code_samples)

    correlations = np.zeros((num_doppler_bins, num_code_phases), dtype=complex)
    for i, doppler_freq in enumerate(doppler_arr_hz):
        if print_progress:
            print(f"\rProcessing Doppler bin {i+1}/{num_doppler_bins}: {doppler_freq} Hz", end="")
        carrier_phase = 2 * np.pi * doppler_freq * time_arr
        conj_carrier_samples = np.exp(-1j * carrier_phase)

        wipeoff_samples = received_samples * conj_carrier_samples

        for j in range(num_code_phases):
            shifted_code_samples = np.roll(code_samples, j)
            correlations[i, j] = np.sum(wipeoff_samples * shifted_code_samples)

        # Frequency-domain correlation
        # wipeoff_samples_fft = np.fft.fft(wipeoff_samples)
        # corr = np.fft.ifft(wipeoff_samples_fft * np.conj(code_samples_fft))
        # correlations[i, :] = corr
    if print_progress:
        print("\nCorrelation complete.")

    return correlations, doppler_arr_hz

In [ ]:
integration_duration = 0.001  # 1 ms
num_coh_samples = int(samp_rate * integration_duration)
min_doppler_hz = -5000
max_doppler_hz = 5000
doppler_step_hz = 1 / integration_duration  # 1 kHz
correlations, doppler_arr_hz = delay_doppler_correlate_gps_l1ca_signal(
    received_samples=quantized_samples[:num_coh_samples],
    prn=10,
    min_doppler_hz=min_doppler_hz,
    max_doppler_hz=max_doppler_hz,
    samp_rate=samp_rate,
    doppler_step_hz=doppler_step_hz,
)

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
num_doppler_bins, num_code_phases = correlations.shape
extent = [0, num_code_phases, min_doppler_hz, max_doppler_hz]

norm_corr_power = np.abs(correlations / num_coh_samples)**2
noise_power = np.median(norm_corr_power)
norm_corr_power = norm_corr_power / noise_power
norm_corr_power_dB = 10 * np.log10(norm_corr_power + 1e-12)

peak_doppler_idx, peak_code_phase_idx = np.unravel_index(
    np.argmax(norm_corr_power_dB),
    norm_corr_power_dB.shape,
)
peak_doppler_hz = doppler_arr_hz[peak_doppler_idx]
print(f"Peak correlation at Doppler: {peak_doppler_hz} Hz, Code Phase: {peak_code_phase_idx} samples")

im = ax.imshow(
    norm_corr_power,
    extent=extent,
    aspect="auto",
    interpolation="nearest",
    cmap="plasma",
    origin="lower",
    vmin=noise_power,
)
code_phase_window = 50  # samples
ax.set_xlim(peak_code_phase_idx - code_phase_window, peak_code_phase_idx + code_phase_window)
ax.set_xlabel("Code Phase [samples]")
ax.set_ylabel("Doppler Frequency [Hz]")
ax.set_title("Delay-Doppler Correlation Map for PRN 10")
plt.colorbar(im, label="Correlation Magnitude [dB]")
plt.show()

# Lecture 11 Exercise 2

With $P_\text{fa} = 10\%$ for a single correlation bin, we want to compute the threshold detection level, and we assume a single 1 ms correlation ($M_1$, $T_\text{coh}=1$ ms).

The lecture 10 slides use the Rayleigh distribution to model the coherent correlation noise, and imply that we should use this to compute the threshold.  If we consider $Y=\sum_{m=1}^M |X_m|^2$ where $X_m$ are independent complex Gaussian random variables with zero mean and variance $\sigma^2$, then $\sqrt{Y}$ follows a Chi-squared distribution with $2M$ degrees of freedom.  Thus, the Rayleigh random variable is the square root of a Chi-squared random variable with 2 degrees of freedom (M=1).

$$
1 - \text{CDF}(\text{threshold}) = P_\text{fa}  \\
\Rightarrow \text{exp}(-y^2 / 2\sigma^2) = P_\text{fa} \\
\Rightarrow y = \sigma \sqrt{-2 \ln(P_\text{fa})}
$$

If $\sigma_\epsilon^2 = 1$ is the variance of the noise samples of the correlation outputs $X_i$, then our threshold is simply $y = \sqrt{-2 \ln(0.1)} \approx 2.146$.

In [ ]:
np.sqrt(-2 * np.log(0.1))

# Lecture 11 Exercise 3



Calculate the probability of detection for a signal with C/N0 values of 20, 25, ..., 50 dB-Hz, assuming 1 ms vs 100 ms coherent integration (M=1) and a false alarm probability of 10%.

The distribution of signal plus noise for $Y=\sum_{m=1}^M |X_m|^2$ is a non-central Chi-squared distribution with 2 degrees of freedom and non-centrality parameter equal to the coherent integration SNR.

The slides use $\sqrt{Y}$, which has a Rician distribution (like a "non-central Rayleigh") with parameters $(\nu, \sigma)$ where $\nu = \sqrt{2 \cdot \text{SNR}}$ and $\sigma^2$ is the variance of the noise samples of the correlation outputs $X_i$.

Note: we define our noise power such that $\sigma^2 = \frac{1}{2}\sigma_\text{Re}^2 = \frac{1}{2}\sigma_\text{Im}^2$ where $\sigma_\text{Re}^2$ and $\sigma_\text{Im}^2$ are the variances of the real and imaginary parts of the complex Gaussian random variable $X_i$.  If you look on wikipedia, the Rician distribution is defined with $\sigma^2$ being the variance of the real and imaginary parts, so we need to be careful when comparing formulas.  Also, the pdf formula in the slides I think was wrong last time I looked (should be $1/\sigma^2$ out front, not $1/\sqrt{\sigma^2}$).  Hopefully this will be updated soon.



Our equivalent formula is:

$$
f(y | \nu, \sigma) = \frac{2y}{\sigma^2} \exp(-\frac{y^2 + A^2}{\sigma^2}) I_0\left(\frac{2y A}{\sigma^2}\right)
$$

where $I_0$ is the modified Bessel function of the first kind of order zero and $A = \sqrt{\text{SNR} \cdot \sigma^2}$.


Plugging in $\text{SNR} = A^2 / \sigma^2$ into our pdf, we get:

$$
f(y | A, \sigma) = \frac{2y}{\sigma^2} \exp\left(-\frac{y^2}{\sigma^2}-\text{SNR}\right) I_0\left(\frac{2y}{\sigma}\sqrt{\text{SNR}}\right)
$$

The CDF of this distribution does not have a closed form, but it can be computed using the Marcum Q-function:

$$
\text{CDF}(y | A, \sigma) = 1 - Q_1\left(\frac{A}{\sigma}, \frac{y}{\sigma}\right)
$$


$$
Q_1(a, b) = \int_b^\infty x \exp\left(-\frac{x^2 + a^2}{2}\right) I_0(ax) \, dx
$$

$$
P_d = 1 - \text{CDF}(\text{threshold} | A, \sigma) = Q_1\left(\frac{A}{\sigma}, \frac{\text{threshold}}{\sigma}\right)
$$

The problem statement implies that we should compute the Marcum Q-function numerically to get the probability of detection.

Below, I just use the `scipy.stats` implementation of the rice distribution CDF to compute the probability of detection.

In [ ]:
from scipy.stats import rice, rayleigh

y = np.arange(0, 10, 0.01)
snr = 5.0  # example SNR value
noise_var = 1.0
sig_ampl = np.sqrt(snr * noise_var)
rayl_pdf_vals = rayleigh.pdf(y, scale=np.sqrt(noise_var))
rice_pdf_vals = rice.pdf(y, b=np.sqrt(snr), scale=np.sqrt(noise_var))

prob_fa = 0.1
threshold = np.sqrt(-2 * noise_var * np.log(prob_fa))


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(y, rayl_pdf_vals, label=f"noise (Rayleigh)", color="blue")
ax.plot(y, rice_pdf_vals, label=f"signal+noise (Rice)", color="orange")
ax.vlines(sig_ampl, 0, np.max(rice_pdf_vals), colors="red", label="A")
ax.vlines(threshold, 0, np.max(rayl_pdf_vals), colors="k", linestyles="dashed", lw=3, label="threshold")
ax.set_xlabel("Amplitude")
ax.set_ylabel("Probability Density Function (PDF)")
ax.set_title("Rice Distribution PDF")
ax.legend()
plt.show()

In [ ]:
cn0_arr = np.arange(20, 51, 5)  # 20 to 50 dB-Hz
false_alarm_prob = 0.1

noise_var = 1.0  # assume noise variance of 1 for correlation outputs
for T_coh in [1e-3, 10e-3, 100e-3]:  # 1 ms and 100 ms
    for i, cn0_dBHz in enumerate(cn0_arr):
        cn0_linear = 10**(cn0_dBHz / 10)
        SNR_coh = cn0_linear * T_coh
        A = np.sqrt(SNR_coh * noise_var)

        threshold = np.sqrt(-2 * noise_var * np.log(false_alarm_prob))

        # Probability of detection using Marcum Q-function
        prob_detection = 1 - rice.cdf(threshold, b=A, scale=np.sqrt(noise_var))
        print(f"T_coh: {T_coh*1e3:.1f} ms, C/N0: {cn0_dBHz} dB-Hz, P_D: {prob_detection:.4f}")

### 